In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from geobr import read_state, read_municipality
from matplotlib.colors import LogNorm
import unicodedata
import os

print("Bibliotecas importadas com sucesso.")

Bibliotecas importadas com sucesso.


In [18]:
def normalizar_log_custom(coluna):
    """
    Normaliza uma coluna usando a escala logarítmica customizada:
    log(x + 1) / log(max(x) + 1)
    """
    # Adiciona +1 para evitar log(0)
    coluna_log = np.log(coluna + 1)
    # Pega o log do valor máximo (também +1)
    max_log = np.log(coluna.max() + 1)
    
    # Evita divisão por zero se o max_log for 0
    if max_log == 0:
        return pd.Series(0, index=coluna.index)
        
    return coluna_log / max_log

In [19]:
# Carrega o shapefile de SP (apenas uma vez)
try:
    mun_sp = read_municipality(code_muni="SP", year=2020)
except Exception as e:
    print(f"Erro ao carregar dados do geobr: {e}. Certifique-se de ter conexão com a internet.")
    # Fallback ou tratamento de erro

def normalizar_string(text):
    """Remove acentos e converte para minúsculas."""
    if isinstance(text, str):
        return unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8").lower()
    return text

# Adiciona coluna normalizada no geodataframe
mun_sp["name_norm"] = mun_sp["name_muni"].apply(normalizar_string)

def plot_choropleth_ranking(df_ranking, coluna_cidade, coluna_score, titulo_mapa, destacar_top=5, escala_log=False):
    """
    Plota um mapa coroplético do estado de SP com base em um score.
    (Versão modificada para aceitar um título)
    """
    # Normaliza os nomes para o merge
    df_ranking["name_norm"] = df_ranking[coluna_cidade].apply(normalizar_string)

    # Junta geodados com ranking
    gdf = mun_sp.merge(df_ranking, on="name_norm", how="left")

    # Garante que não tenha zero se for usar log
    gdf[coluna_score] = gdf[coluna_score].fillna(0)
    if escala_log:
        gdf[coluna_score] = gdf[coluna_score].replace(0, 1)

    fig, ax = plt.subplots(figsize=(12, 12))

    # Define estilo do mapa
    norm = LogNorm(vmin=gdf[coluna_score].min(), vmax=gdf[coluna_score].max()) if escala_log else None
    legenda_label = f"{coluna_score} (escala log)" if escala_log else coluna_score

    # Desenha o mapa
    gdf.plot(
        column=coluna_score,
        cmap="OrRd",
        linewidth=0,
        ax=ax,
        norm=norm,
        legend=True,
        legend_kwds={"label": legenda_label}
    )

    # Borda do estado
    read_state("SP", year=2020).plot(ax=ax, facecolor="none", edgecolor="black", linewidth=2)

    # Destaca top municípios
    if destacar_top > 0:
        top = gdf.nlargest(destacar_top, coluna_score)
        top.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=2)

        # Labels
        print(f"--- Top {destacar_top} Cidades (Score: {coluna_score}) ---")
        for _, row in top.iterrows():
            c = row.geometry.centroid
            print(f"{row[coluna_cidade]} (Score: {row[coluna_score]:.4f})")

    ax.set_title(titulo_mapa, fontsize=16)
    ax.axis("off")
    
    # Salvar a figura em vez de mostrar
    plt.savefig(f"{titulo_mapa.replace(' ', '_').replace('(', '').replace(')', '')}.png", bbox_inches='tight')
    plt.close(fig) # Fecha a figura para economizar memória
    print(f"Mapa salvo como: {titulo_mapa.replace(' ', '_').replace('(', '').replace(')', '')}.png")

In [20]:
# Definindo o caminho base
DATA_PATH = './dados/exportados/' # Assumindo que os arquivos estão na raiz ou em subpastas

# Verificando os caminhos corretos
# (Usando os nomes dos arquivos que você forneceu)
path_pop = DATA_PATH + 'DimPopulacao.csv'
path_farm = DATA_PATH + 'DimContagemFarmacias.csv'
path_pib = DATA_PATH + 'DimPIB.csv'
path_cidade = DATA_PATH + 'DimCidadePotencial.csv'
path_pesos = DATA_PATH + 'DimPesoFaixaEtaria.csv'

# Carregar dados das dimensões
try:
    DimPopulacao = pd.read_csv(path_pop, sep=';')
    DimContagemFarmacias = pd.read_csv(path_farm, sep=';')
    DimPIB = pd.read_csv(path_pib, sep=';')
    DimCidadePotencial = pd.read_csv(path_cidade, sep=';')
    DimPesoFaixaEtaria = pd.read_csv(path_pesos, sep=';')
    
    print("Arquivos CSV de dimensões carregados com sucesso.")
except FileNotFoundError as e:
    print(f"Erro: Arquivo não encontrado. Verifique o caminho: {e}")

Arquivos CSV de dimensões carregados com sucesso.


In [21]:
# 1. Pré-processar População
# Calcula População Total por cidade
pop_total_por_cidade = DimPopulacao.groupby('codigoibge')['contagempop'].transform('sum')
DimPopulacao['PopTotal'] = pop_total_por_cidade

# Filtra cidades (ex: > 99k habitantes)
df_pop_filtrado = DimPopulacao[DimPopulacao['PopTotal'] > 99000].copy()

# Renomeia colunas para o merge com a tabela de pesos (MUITO IMPORTANTE)
df_pop_pronto = df_pop_filtrado.rename(columns={
    'faixaetariapop': 'faixaetaria',
    'sexopop': 'sexofaixaetaria'
})
# Seleciona colunas úteis
df_pop_pronto = df_pop_pronto[['codigoibge', 'contagempop', 'faixaetaria', 'sexofaixaetaria', 'PopTotal']]


# 2. Pré-processar PIB
df_pib_pronto = DimPIB[
    (DimPIB['setorpib'] == 'Comércio e Serviços') & 
    (DimPIB['anopib'] == 2021)
][['codigoibge', 'valorpib']]

print("Dados de População e PIB pré-processados.")

Dados de População e PIB pré-processados.


In [27]:
# 1. Pega a lista de estimativas únicas
lista_estimativas = DimPesoFaixaEtaria['nomeestimativa'].unique()
print(f"Encontradas {len(lista_estimativas)} estimativas: {lista_estimativas}")

# 2. Loop principal
for estimativa in lista_estimativas:
    print(f"\n--- Iniciando Cálculo para: {estimativa} ---")
    
    # --- Passo A: Filtrar Pesos da Estimativa Atual ---
    df_pesos_atual = DimPesoFaixaEtaria[
        DimPesoFaixaEtaria['nomeestimativa'] == estimativa
    ][['faixaetaria', 'sexofaixaetaria', 'valorpeso']]
    
    # --- Passo B: Calcular Score Ponderado (com MERGE) ---
    df_pop_ponderado = pd.merge(
        df_pop_pronto,
        df_pesos_atual,
        on=['faixaetaria', 'sexofaixaetaria'],
        how='inner' 
    )
    df_pop_ponderado['score_ponderado_linha'] = df_pop_ponderado['contagempop'] * df_pop_ponderado['valorpeso']
    
    # --- Passo C: Agregar Score de População por Cidade ---
    df_score_pop_agg = df_pop_ponderado.groupby('codigoibge').agg(
        score_pop_raw=('score_ponderado_linha', 'sum'),
        PopTotal=('PopTotal', 'first')
    ).reset_index()
    
    # --- Passo D: Montar Tabela Final (FatoScore) ---
    df_final = df_score_pop_agg.merge(
        DimCidadePotencial[['codigoibge', 'nomecidadepotencial']],
        on='codigoibge',
        how='left'
    ).merge(
        DimContagemFarmacias[['codigoibge', 'qtdfarmacias']],
        on='codigoibge',
        how='left'
    ).merge(
        df_pib_pronto,
        on='codigoibge',
        how='left'
    )
    
    df_final['qtdfarmacias'] = df_final['qtdfarmacias'].fillna(0)
    df_final['valorpib'] = df_final['valorpib'].fillna(0)
    
    # --- Passo E: Calcular Scores (Saturação e PIB) ---
    df_final["score_saturacao_raw"] = df_final.apply(
        lambda r: r["score_pop_raw"] if r["qtdfarmacias"] == 0 else r["score_pop_raw"] / r["qtdfarmacias"],
        axis=1
    )
    df_final["score_pib_raw"] = df_final.apply(
        lambda r: 0 if r["PopTotal"] == 0 else r["valorpib"] / r["PopTotal"],
        axis=1
    )
    
    # --- Passo F: Normalizar e Calcular Score Final ---
    df_final["score_pop_norm"] = normalizar_log_custom(df_final["score_pop_raw"])
    df_final["score_saturacao_norm"] = normalizar_log_custom(df_final["score_saturacao_raw"])
    df_final["score_pib_norm"] = normalizar_log_custom(df_final["score_pib_raw"])

    peso_pop = 1/10
    peso_saturacao = 7/10
    peso_pib = 2/10

    df_final["score_final"] = (
        df_final["score_pop_norm"] * peso_pop +
        df_final["score_saturacao_norm"] * peso_saturacao +
        df_final["score_pib_norm"] * peso_pib
    )
    df_final["score_final_norm"] = normalizar_log_custom(df_final["score_final"])
    
    # --- Passo G: Ordenar, Printar (NOVO) e Plotar ---
    df_ranking_final = df_final.sort_values("score_final_norm", ascending=False)
    
    print(f"\n--- Top 5 Cidades para '{estimativa}' ---")
    
    # ----> MUDANÇA AQUI <----
    # Printando as colunas mais relevantes do top 5
    print(df_ranking_final[
        [
            'nomecidadepotencial', 
            'score_final_norm', 
            'score_pop_norm', 
            'score_saturacao_norm', 
            'score_pib_norm',
            'PopTotal',
            'qtdfarmacias'
        ]
    ].head())
    
    # Gerar o plot
    plot_choropleth_ranking(
        df_ranking_final,
        coluna_cidade='nomecidadepotencial',
        coluna_score='score_final_norm',
        titulo_mapa=f"Ranking Final - {estimativa}",
        destacar_top=10 # O plot ainda destaca 10 no mapa
    )

print("\n--- Processo concluído ---")
print("Todos os mapas de estimativa foram gerados e salvos em arquivos .png.")
print("Os DataFrames do Top 5 para cada estimativa foram exibidos no console.")

Encontradas 3 estimativas: ['Uso de medicamentos (Doenças Crônicas)' 'Prioridade Idosos (60+)'
 'Peso Uniforme (Todos)']

--- Iniciando Cálculo para: Uso de medicamentos (Doenças Crônicas) ---

--- Top 5 Cidades para 'Uso de medicamentos (Doenças Crônicas)' ---
      nomecidadepotencial  score_final_norm  score_pop_norm  \
66              São Paulo          1.000000        1.000000   
45                 Osasco          0.994431        0.860407   
17                Cubatão          0.992930        0.765317   
47               Paulínia          0.992070        0.766140   
61  São Bernardo do Campo          0.990114        0.867845   

    score_saturacao_norm  score_pib_norm  PopTotal  qtdfarmacias  
66              0.981424        0.906530  11451999          4887  
45              0.978356        0.950019    728615           310  
17              0.999651        0.913072    112476            36  
47              0.973068        1.000000    110537            50  
61              0.988068